<a href="https://colab.research.google.com/github/marketconnect/notebooks/blob/main/top_selling_items.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json
import getpass

import os

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read week orders

In [ ]:
tsi_path = '/content/drive/MyDrive/parsing/input/tsi.csv'



tsi_df = pd.read_csv(tsi_path)
print(len(tsi_df))
tsi_df.head()

169850


,sku,total_orders,total_revenue,id,subjectId,name,supplier,reviewRating,feedbacks
0,184032884,1420,281985100,184032884.0,11.0,Утепленные брюки палаццо школьные,Fart Favorita,4.5,36278.0
1,24695842,1979,263735500,24695842.0,11.0,Брюки бананы укороченные 7 8 на резинке Лаина,LAINA & STELLIO,4.7,114265.0
2,251838531,1072,232249400,251838531.0,11.0,Брюки утепленные палаццо,FSV,4.7,1772.0
3,73378778,1335,222305300,73378778.0,11.0,Школьные брюки палаццо,Fart Favorita,4.6,53295.0
4,85423567,991,217712900,85423567.0,11.0,Брюки клеш от колена,GLAUREN,4.6,9740.0


In [ ]:
preoritize_baskets = [0, 10075871, 14991731, 30318442, 46107348, 100758471, 102969644, 108869822, 114623766, 120056228, 155414202, 161954586, 185063168, 197368454, 214752094, 222672407, 255863248]

def get_preoritized_basket(product_id):

    for i in range(1, len(preoritize_baskets)):
        if preoritize_baskets[i] > product_id:
            return i
    if preoritize_baskets[-1] <= product_id:
        return len(preoritize_baskets)
    return -1

def find_valid_image_url(product_id):

    vol = f"vol{product_id // 100000}"
    part = f"part{product_id // 1000}"

    preoritized_basket_num = get_preoritized_basket(product_id)

    if preoritized_basket_num > 1:
        baskets = [preoritized_basket_num - 1, preoritized_basket_num]
        for b_num in range(baskets[0], baskets[1] + 1):
            url = f"https://basket-{b_num:02d}.wbbasket.ru/{vol}/{part}/{product_id}/images/c246x328/1.webp"

            try:
                resp = requests.get(url)
                if resp.status_code == 200:
                    return url
            except requests.RequestException as e:
                return f"Failed to make GET request: {str(e)}"


    for basket_num in range(1, 17):
        url = f"https://basket-{basket_num:02d}.wbbasket.ru/{vol}/{part}/{product_id}/images/c246x328/1.webp"

        try:
            resp = requests.get(url)
            if resp.status_code == 200:
                return url
        except requests.RequestException as e:
            return f"Failed to make GET request: {str(e)}"

    return f"No valid URL found for product ID {product_id}"


In [ ]:
def process_data_in_chunks(input_file, output_file, chunk_size=10000):
    if os.path.exists(output_file):
        processed_df = pd.read_csv(output_file)
        processed_skus = set(processed_df['sku'].tolist())
        mode = 'a'
    else:
        processed_df = pd.DataFrame()
        processed_skus = set()
        mode = 'w'


    chunk_iter = pd.read_csv(input_file, chunksize=chunk_size)

    for chunk in chunk_iter:

        chunk_to_process = chunk[~chunk['sku'].isin(processed_skus)]

        if not chunk_to_process.empty:

            chunk_to_process['img'] = chunk_to_process['sku'].apply(find_valid_image_url)


            chunk_to_process.to_csv(output_file, mode=mode, header=(mode == 'w'), index=False)
            mode = 'a'

            print(f"Processed {len(chunk_to_process)} records and saved to {output_file}")

        processed_skus.update(chunk_to_process['sku'].tolist())



output_file = '/content/drive/MyDrive/parsing/output/tsi_df_with_images.csv'


process_data_in_chunks(tsi_path, output_file, chunk_size=10000)

Processed 10000 records and saved to /content/drive/MyDrive/parsing/output/tsi_df_with_images.csv
Processed 10000 records and saved to /content/drive/MyDrive/parsing/output/tsi_df_with_images.csv
